In [ ]:
!pip install -Uqq fastbook ipywidgets nbdev

import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
# your drive home will be at /content/gdrive/MyDrive/
# your notebooks might be at /content/gdrive/MyDrive/Colab Notebooks/
BASE_DIR = "/content/gdrive/MyDrive/Colab Notebooks/"

In [ ]:
# https://wandb.ai/wandb/common-ml-errors/reports/How-To-Use-GPU-with-PyTorch---VmlldzozMzAxMDk
# torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)

# Convert MNIST data in folders to the format we need
# 1. Open each image file as an Image object and convert it to tensors
# 2. Convert a list of 2D tensors to a 3D tensor using stack(), normalise
#     by converting each value to a float and dividing by max pixel value 255
# 3. Concatenate all the tensors of 3s and 7s together and make 28x28 arrays
#     into a 28*28 length list
# 4. Creatae the labels such that wew have 1 for image '3' and 0 for image '7'

def filesToStackedTensors(d1, d2):
  t = [tensor(Image.open(f)).to(device) for f in (path/d1/d2).ls().sorted()]
  return torch.stack(t, 0).float()/255

def x_y(stacked_x, stacked_y):
  x = torch.cat([stacked_x, stacked_y]).view(-1, 28*28)
  y = tensor([1]*len(stacked_x) + [0]*len(stacked_y)).unsqueeze(1).to(device)
  return (x, y)

In [ ]:
# ??torch.cat
# ??torch.view
# ??torch.unsqueeze

a = torch.ones(4, 1)
b = torch.full((1, 4, 3), 3)
# a, b, torch.cat([a, b]), torch.cat([a, b]).view(-1, 12)

# valid values for unsqueeze in this example: -2 to 1
# tensor([1]*13 + [0]*4).unsqueeze(1)
# tensor([1]*13 + [0]*4).unsqueeze(0)

# ??torch.sum
a, 1 - a.mean()

In [ ]:
train_x, train_y = x_y(filesToStackedTensors('train', '3'), \
                       filesToStackedTensors('train', '7'))
valid_x, valid_y = x_y(filesToStackedTensors('valid', '3'), \
                       filesToStackedTensors('valid', '7'))
# train_x.shape, train_y.shape, valid_x.shape, valid_y.shape
# print(f'x cuda? {train_x.is_cuda}')
# print(f'y cuda? {train_y.is_cuda}')

In [ ]:
def predictions(x, w, b):
  print(f'w: {w[0:1, 0:1]}')
  r = torch.matmul(x, w) + b
  print(f'r.grad_fn: {r.grad_fn}')
  # print(f'r shape: {r.shape}, r: {r[0:4, 0:4]}')
  sums = (r.sum(1)).unsqueeze(1)
  print(f'sums.grad_fn: {sums.grad_fn}')
  # print(f'sums shape: {sums.shape}, sums: {sums[0:4]}')
  sig = torch.sigmoid(sums)
  print(f'sig.grad_fn: {sig.grad_fn}')
  # print(f'sig shape: {sig.shape}, sig: {sig[0:4]}')
  # print(f'sig: {sig[0:4]}, sig.sum: {sig.sum()}')
  return sums

def loss(preds, y):
  # print(f'preds: {preds[0:5]}')
  # print(f'labels: {y[0:5]}')
  # print(f'x shape: {preds.shape}, y shape: {y.shape}')
  a = (preds - y).abs().mean()
  print(f'a.grad_fn: {a.grad_fn}')
  # print(f'a: {a}')
  return a

# loss(predictions(train_x, w, b), train_y)

def apply_step(i, x, y, w, b, lr):
  preds = predictions(x, w, b)
  # print(f'preds.grad_fn: {preds.grad_fn}')
  # l = preds.backward()
  l = loss(preds, y)
  # print(f'l.grad_fn: {l.grad_fn}')
  l.backward()
  # print(f'w after backward: {w[0:1]}')
  print(f'w grads after backward: {w.grad.shape}, {w.grad[0:7, 0:3]}')
  # print(f'w.grad.data: {w.grad.data[0:3]}')
  # print(f'before b.data: {b.data}')
  # print(f'before b.grad: {b.grad.data}')
  w.data = w.data - (w.grad.data * lr)
  b.data = b.data - (b.grad.data * lr)
  print(f'after b.data: {b.data}')
  w.grad = b.grad = None
  return (w, b, l)

def apply_steps(n, x, y, w, b, lr):
  results = []
  for i in range(n):
    # print(f'before b: {b[0:5]}')
    (w, b, l) = apply_step(i, x, y, w, b, lr)
    results.append(l)
    if(i % 1 == 0): print(f'loss at step({i}): {l}')
    # print(f'after b: {b[0:5]}')
  return (w, b, results)

In [ ]:
g = torch.Generator()
g.manual_seed(5)
w = torch.randn((784, 1), generator=g).to(device).requires_grad_(True)
# print(f'w: {w[0:1, 0:1]}')
# print(f'w cuda? {w.is_cuda}')
# print(f'w shape: {w.shape}, w: {w[0:4, 0:4]}')
b = torch.randn(12396, generator=g).to(device).requires_grad_(True)
# print(f'b cuda? {b.is_cuda}')
# print(f'len(b): {len(b)}, b: {b[0:9]}')

# lrs = [0.001, 0.1, 1, 100, 10000, 100000]
lrs = [1]
for lr in lrs:
  print(f'lr: {lr}')
  fw, fb, fr = apply_steps(1, train_x, train_y, w, b, lr)

In [ ]:
#len(range(30)), len(results)
#plt.scatter(range(30), to_np(fr))
p = predictions(train_x, fw, fb)
type(p), len(p), p.shape
diff = (p <= 0.5).float().sum()
# type(diff), len(diff), diff.shape, diff
# diff, len(train_y), loss(p, train_y)

# p[0], train_y[0], p[0] - train_y[0]
diffs = torch.nonzero((p < 0.5).float().unbind(1)[0])
trx = [Image.open(f) for f in (path/'train'/'3').ls().sorted()] + [Image.open(f) for f in (path/'train'/'7').ls().sorted()]

# try = [Image.open(f) for f in (path/'train'/'7').ls().sorted()]
def images():
  for d in diffs[-10:-5]:
    print(f'd: {d}, prediction: {p[d]}')
    show_image(trx[d])

  for i in range(5,10):
    print(f'i: {i}, prediction: {p[i]}')
    show_image(trx[i])

In [ ]:
#??torch.nonzero
t = tensor([0,1,0,2,2,3,0,1,1,1,1])
print(f'nonzero: {torch.nonzero(t)}')

#??torch.linspace
a = torch.linspace(1, 9, steps=9).unsqueeze(1)
a, a[0], a[1], a[0][0],a[0:3]
a = np.array([[1, 2, 3, 4],
              [5, 6, 7, 8],
              [9, 10, 11, 12],
              [13, 14, 15, 16]])
a[0:3][0:3][0:3], a[0:3, 0:3]
a.sum(), a.sum(0), a.sum(1)

In [ ]:
print(len(train_x), train_x.shape, type(train_x), len(train_x[0]), '\n', train_x[0:3, 0:7])
print(len(train_y), train_y.shape, type(train_y), len(train_y[0]), train_y[0:3])
print(len(w), w.shape, type(w), len(w[0]), w[0:2, 0:3])
len(b), type(b), b[0:7]

In [ ]:
a = torch.ones((4,1), requires_grad=True)
print(f'a: {a}')
print(f'a.grad_fn: {a.grad_fn}')
b = a + 1
print(f'b.grad_fn: {b.grad_fn}')
c = a.sum(1)
print(f'c.grad_fn: {c.grad_fn}')
print(f'c: {c}')
d = c.unsqueeze(1)
print(f'd.grad_fn: {d.grad_fn}')
print(f'd: {d}')

In [ ]:
train_x.shape, w.shape